<a href="https://colab.research.google.com/github/sushantjha78/meme-notes/blob/master/meme-notes-cnn-keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
train_test_ratio = 0.92
img_width, img_height = 150, 200
epochs = 5
batch_size = 64

In [26]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from random import shuffle, choice
import numpy as np
import os
import cv2
from PIL import Image
from keras.preprocessing import image

In [3]:

def load_images_resize_bw(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)/1.0
            img = cv2.resize(img, dsize=(150, 200), interpolation=cv2.INTER_CUBIC)
            img = np.array(img)
            images.append(img)
    return images

def label_images(image_list, value):
    labelled_image = []
    for i in range(len(image_list)):
        labelled_image.append(np.append(image_list[i], value))
    return labelled_image



In [4]:

notes = load_images_resize_bw('/content/drive/My Drive/Stage_1/Stage_1/Notes')
memes = load_images_resize_bw('/content/drive/My Drive/Stage_1/Stage_1/Memes')

In [7]:
notes = np.array(notes)
memes = np.array(memes)
notes_labels = np.ones(notes.shape[0])
memes_labels = np.zeros(memes.shape[0])

X = np.concatenate((notes,memes))
Y = np.concatenate((notes_labels,memes_labels))

np.random.seed(0)
np.random.shuffle(X)
np.random.seed(0)
np.random.shuffle(Y)

In [21]:

nb_train_samples = int(train_test_ratio*len(X))
nb_test_samples = len(X) - int(train_test_ratio*len(X))
x_train = X[:nb_train_samples]
x_train = x_train.reshape((x_train.shape[0], img_height, img_width, 1))
x_test = X[nb_test_samples:]
x_test = x_test.reshape((x_test.shape[0], img_height, img_width, 1))
y_train = Y[:nb_train_samples]
y_test = Y[nb_test_samples:]


In [22]:

if K.image_data_format() == 'channels_first':
  input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

datagen = ImageDataGenerator(
    rescale = 1.0/255,
    zoom_range = 0.2,
    horizontal_flip = True)

train_data_generator = datagen.flow(
    x_train,###
    y_train,
    batch_size = batch_size)

test_data_generator = datagen.flow(
    x_test,###
    y_test,
    batch_size = batch_size)


In [33]:
model = Sequential()
model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))



model.compile(loss = 'binary_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])


In [38]:
model.fit_generator(
    train_data_generator,                                                       #data_generator
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    validation_data = test_data_generator,                                      #data_generator
    validation_steps = nb_test_samples // batch_size)

model.save_weights('model-1.h5')


Epoch 1/5
23/23 [==============================] - 34s 1s/step - loss: 0.9180 - accuracy: 0.7317 - val_loss: 0.2076 - val_accuracy: 0.9453
Epoch 2/5
23/23 [==============================] - 33s 1s/step - loss: 0.2745 - accuracy: 0.9103 - val_loss: 0.1600 - val_accuracy: 0.9609
Epoch 3/5
23/23 [==============================] - 33s 1s/step - loss: 0.1784 - accuracy: 0.9490 - val_loss: 0.2078 - val_accuracy: 0.9453
Epoch 4/5
23/23 [==============================] - 36s 2s/step - loss: 0.1073 - accuracy: 0.9715 - val_loss: 0.0654 - val_accuracy: 0.9844
Epoch 5/5
23/23 [==============================] - 34s 1s/step - loss: 0.2656 - accuracy: 0.9192 - val_loss: 0.1611 - val_accuracy: 0.9375
